In [1]:
from torch.utils.data import DataLoader
from src.dataset import FrameDataset, custom_collate
from sklearn.preprocessing import LabelEncoder
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import torch
from src.averager import Averager


In [2]:
le = LabelEncoder()
le.fit(['apadding', 'bicycle','human', 'motorcycle', 'vehicle'])
le.classes_


array(['apadding', 'bicycle', 'human', 'motorcycle', 'vehicle'],
      dtype='<U10')

In [3]:
training_data = FrameDataset(root = 'data', label_encoder = le)
train_dataloader = DataLoader(training_data, batch_size=1, shuffle=True, collate_fn=custom_collate)
train_features, targets = next(iter(train_dataloader))

In [4]:
# load a model; pre-trained on COCO
model = fasterrcnn_resnet50_fpn(num_classes=5)

In [5]:
num_classes = 5  # 4 classes + background/padding
device = torch.device('cuda')

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# # replace the pre-trained head with a new one
# model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# # Load the trained weights
# model.load_state_dict(torch.load(WEIGHTS_FILE))  ##Load pre trained weights
# #model.eval()

# x = model.to(device)

In [6]:
model.train()
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.01, momentum=0.9, weight_decay=0.00001)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)
lr_scheduler = None

num_epochs = 5

loss_hist = Averager()
itr = 1

for epoch in range(num_epochs):
    loss_hist.reset()
    
    for frames, targets in train_dataloader:
        
        frames = [frame.to(device) for frame in frames]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(frames, targets)   ##Return the loss

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        loss_hist.send(loss_value)  #Average out the loss

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if itr % 50 == 0:
            print(f"Iteration #{itr} loss: {loss_value}")

        itr += 1
    
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

    print(f"Epoch #{epoch} loss: {loss_hist.value}")

Iteration #50 loss: nan


KeyboardInterrupt: 